## What is a Genetic Algorithm?

It is a optimization algorithm inspired on the ideia of Evolution of Species through Natural Selection.
There basic parts of it are::
- Initial population: set of individuals wtih different caractheristics.
- Fitness function: a way to evaluate how fitted for survival each indidual is.
- Selection: step of selecting the fittest individuals that will pass genes to the next generation.
- Crossover: mixture of the genes from each parent individual.
- Mutation: random mutations that occur in the offspring



A pseudo algorithm could be:
1. set convergence rule
1. current population = initial population
1. while not converged: 
    1. calculate fitness score of each individual in current population
    1. select fittest
    1. set offspring = crossover of fittest
    1. do mutation step to offspring
    1. current population = offspring


In [1]:
import numpy as np
import pandas as pd
from string import ascii_uppercase
import random

In [149]:
# each individual is a string of letters
POP_SIZE = 100
GENES = "helowrd"#ascii_uppercase
GENES_SIZE = len(GENES)
SOLUTION = "helloworld"#.upper()
INDIVIDUAL_SIZE = len(SOLUTION)

SELECTION_PROP = 0.20
SELECTION_N = int(POP_SIZE * SELECTION_PROP)

MUTATION_PROP = 0.1

In [3]:
def get_random_individual():
    return ''.join(random.choices(GENES, k = INDIVIDUAL_SIZE))



def get_fitness_score(individual: str):
    # compare each gene
    y = [individual[i] == SOLUTION[i] for i in range(INDIVIDUAL_SIZE)]
    return 1 + np.sum(y)



def select_fittests(curr_scores):
    idx_order = np.argsort(-curr_scores)[:SELECTION_N]
    return idx_order

def select_pairs(curr_scores, idx_order):

    # select random individual with prob proportional to score
    sel_scores = curr_scores[idx_order]
    weights = sel_scores / np.sum(sel_scores)

    idx_parents1 = np.random.choice(idx_order, size=POP_SIZE, replace=True, p=weights)
    idx_parents2 = np.random.choice(idx_order, size=POP_SIZE, replace=True, p=weights)
    
    return idx_parents1, idx_parents2



def crossover_pair(x, y):
    # returns a new individual
    if x==y: return x
    choice = np.random.binomial(n=1,p=0.5, size=INDIVIDUAL_SIZE)
    return ''.join([x[i] if choice[i] else y[i] for i in range(INDIVIDUAL_SIZE)])



def crossover(parents1, parents2):
    return np.array([crossover_pair(x,y) for x, y in zip(parents1, parents2)])


def mutate_individual(x):
    mutate = np.random.binomial(n=1,p=MUTATION_PROP, size=INDIVIDUAL_SIZE)
    return ''.join([GENES[np.random.randint(0, GENES_SIZE)] if mutate[i] else x[i] for i in range(INDIVIDUAL_SIZE)])


def mutate(pop):
    return np.array([mutate_individual(x) for x in pop])



In [42]:

initial_pop = np.array([get_random_individual() for i in range(POP_SIZE)])
initial_pop[:10]

array(['ZNWOQMUBSN', 'LOYFFTCQNZ', 'PUVJNIVLEV', 'LRFTROGNYJ',
       'LUSYZWVWLM', 'VXXJDKYOLA', 'VIDXKNONPY', 'CODREZHMHY',
       'QDLNNGDFGM', 'CIGBNTLWIN'], dtype='<U10')

In [43]:
from tqdm.cli import tqdm

In [44]:
curr_pop = initial_pop

GEN_LIMIT = 1000
best_scores = []
best_individuals = []

for i in tqdm(range(GEN_LIMIT)):
    # 1. calculate fitness score of each individual in current population
    curr_scores = np.array([get_fitness_score(x) for x in initial_pop])

    # 1. select fittest
    idx_order = select_fittests(curr_scores)
    best_scores.append(curr_scores[idx_order[0]])
    best_individuals.append(curr_pop[idx_order[0]])

    # 1. set offspring = crossover of fittest
    idx_parents1, idx_parents2 = select_pairs(curr_scores, idx_order)
    offspring = crossover(curr_pop[idx_parents1], curr_pop[idx_parents2])

    # 1. do mutation step to offspring
    offspring = mutate(offspring)

    # 1. current population = offspring
    curr_pop = offspring

100%|██████████| 1000/1000 [00:28<00:00, 35.03it/s]
